# Gas transport equations and utility functions

#### Version1.2      11/Nov/2019, Chris Thompson           Status  
We have implemented formulae for three sets of steady gas transport equations: Simplified Isothermal, AGA and Weymouth; this includes formule for the transmission factors. For the Weymouth formulae we have implemented a formula which returns the undefined element of the defining triple: $ \left < \dot{Q_{st}}, P_{in}, P_{out} \right >$.
1. There is some qualitative validation. This could be compared with `PipeSim` or `PIPE-FLO`.
2. The debugging output needs to be cleaned up.
   1. We need to balance readability and compactness.
3. We use  linear  interpolation as an equation solver, this should be replaced:
      1. with `scipy.optimize.solve()` **- in progress.**
      2. with (possibly) Bayesian Optimisation packages:
         1. `https://github.com/fmfn/BayesianOptimization` **- in progress.**
         2. `https://github.com/SheffieldML/GPyOpt`
3. We solved the three cases in the worksheet (assuming flow is horizontal, isothermal flow). This should be generalised.
   1. Check the implementation of the potential energy term $E_{PE}$
   2. Implement thermal effects (possibly in stages) - we need properties, `Equation of State function` (see Gas Viscosity and Equations of State.ipynb).
   3. Include compressibility ($Z$) effects.
   4. Include uncertainty effects (physical properties and measurement error).
1. We need to extend the application to use other than Weymouth.
1. What information do we supply to support learning?
   1. Some figures in one of the formulae?
   2. One case worked in detail?
   3. Something about Kirchoff's laws?
   4. Some intermediate or partial results? Posted
1. There should be more examples in the `PIPE-FLO` and `PIPESIM` materials.
1. Further cases can be found in Mohitpour P118 (Second Edition) and in the exam.
4. Look at leaks.
   5. Use BayesOpt to handle solution in the presence of uncertainty.
   5. Look at $\Sigma$Flow as a tool.
   5. Sort out PIPESIM + Python.


### Introduction

 We have derived the steady transport equation for gas along a pipeline:
 >
 $
 \left ( \frac{464 S_g}{\pi^2 \overline{R}^2}\right )
  \left ( \frac{P_{st}}{z_{st}T_{st}}\right )^2 
   \frac{fL}{D^5}\dot{Q}_{st}^2 =
  \frac{P_{in}^2-P_{out}^2}{z_{avg} \overline{R} T_{avg}}
 + \frac{g P^2_{avg}58 S_g}{z^2_{avg} \overline{R}^2 T^2_{avg}}(H_{in}-H_{out}) 
 $

### A Selection of Transport Equations
 <font color = blue>We can re-arrange this to give:</font>
- The Simplified Isothermal transport equation:  
> $
\dot{Q}_{st} = 13.2986 \frac{T_{st}}{P_{st}}
\left [ \frac{P_1^2-P_2^2-E_{PE}}{f L ~S_g~ T_{avg}z_{avg}} \right ] ^{1/2}
D^{5/2}
$
.
- The AGA fully turbulent gas transport equation:    
> $
\dot{Q}_{st} = 13.2986 \frac{T_{st}}{P_{st}}
\left [ \frac{P_1^2-P_2^2-E_{PE}}{L ~S_g~ T_{avg}z_{avg}} \right ] ^{1/2}
\left [ 2\log_{10}\frac{3.7 D}{\epsilon}\right ] D^{5/2}
$
- We can derive the Weymouth equation; valid for fully turblulent flows    
> $
\dot{Q}_{st} = 137.2364 \frac{T_{st}}{P_{st}}
\left [ \frac{P_1^2-P_2^2-E_{PE}}{L ~S_g~ T_{avg}z_{avg}} \right ] ^{1/2}
D^{8/3}
$

### Different transmission factors
-  `TFWey`: The transmission factor for the Weymouth equation is:
> $
\frac{1}{\sqrt{f}} = 10.3196 D^{\frac{1}{6}}
$
- `TFAGA`: The transmission factor used for the AGA equation is:
> $
\frac{1}{\sqrt{f}} = 2 \log_{10}\left (\frac{3.7D}{\epsilon}\right )
$
- `TFColebrook`: The Colebrook-White formula for transmission factor is:
> $
\frac{1}{\sqrt{f}} = -2.0 \log_{10} \left [ \frac{\epsilon}{3.7D} + \frac{2.51}{Re \sqrt{f}}\right ]
$
-  `TFTEST` is supplied for testing purposes only!

### Validation

#### Validation -- transmission factors

In [1]:
import math
#Define the transmission factors
def TFColebrook(D, EPS, Reynolds = 1E6, reltol = 1.0E-5, DEBUG = 0):
    """
    From Crain P6-2 Eqn 6-5
    Valid for 5,000< Re < 3E8 and 1E-6 < EPS/D < 0.01
    """
    #Initialise with Weymouth TF
    TFCOld = 10.3196 * pow(D,1/6)
    for i in range (0,20):
        LT1   = EPS/(3.7*D)
        LT2   = (2.51/Reynolds)*TFCOld
        LTERM = LT1 + LT2
        TFCNew= -2*math.log10(LTERM)
        if (DEBUG == 1) : print("i: ",i, "TFCOld: ", TFCOld," TFCNew: ", TFCNew," LTERM: ", LTERM)
        DeltaTFC = abs(TFCNew-TFCOld)
        TFCOld = TFCNew; 
        if abs(DeltaTFC/TFCNew) < reltol :
            if DEBUG == 1 : 
                print("CW implicit; convergence? Delta TFColebrook:", '{:12.3e}'.format(abs(TFCNew-TFCOld)), 
                  " TFCNew: ", '{:12.2e}'.format(TFCNew), 
                  " LogT1 ", '{:12.4e}'.format(LT1) , " LogT2 ", '{:12.4e}'.format(LT2))
            return TFCNew
    print("Failure to converge!")
    print("i: ",i, "TFCNew: ", '{:12.4e}'.format(TFCNew)," TFCOld: ", '{:12.4e}'.format(TFCOld))
    return -99999

def TFWey(D, EPS, Reynolds = 1E6):
    TFW = 10.3196 * pow(D,1/6)
    return TFW

def TFSJain(D, EPS, Reynolds = 1E6):
    """
    From Crain P6-2 Eqn 6-7
    Valid for 5,000< Re < 3E8 and 1E-6 < EPS/D < 0.01
    (because TF = 1/sqrt(f))
    """
    LTERM = EPS/(3.7*D) + 5.74/pow(Reynolds,0.9)
    TFSJ= 2*math.log10(LTERM)
    return TFSJ

def TFAGA(D, EPS, Reynolds = 1E6, reltol = 1.0E-5, DEBUG = 0):
    """
    From Coelho and Pinho
    """
    #Initialise with Weymouth TF
    TFAGAOld = 10.3196 * pow(D,1/6)
    for i in range (0,20):
        LTERM = (2.825/Reynolds)*TFAGAOld
        TFAGANew= -2*math.log10(LTERM)
        if (DEBUG == 1) : print("i: ",i, "TFAGAOld: ", TFAGAOld," TFAGANew: ", TFAGANew," LTERM: ", LTERM)
        DeltaTFAGA = abs(TFAGANew-TFAGAOld)
        TFAGAOld = TFAGANew; 
        if abs(DeltaTFAGA/TFAGANew) < reltol :
            #print("AGA implicit; convergence? Delta TFAGA:", '{:12.3e}'.format(abs(TFAGANew-TFAGAOld)), " TFAGANew: ", '{:12.2e}'.format(TFAGANew), " LogTERM ", '{:12.4e}'.format(LTERM))
            return TFAGANew
    print("Failure to converge!")
    print("i: ",i, "TFAGANew: ", '{:12.4e}'.format(TFAGANew)," TFCOld: ", '{:12.4e}'.format(TFAGAOld))
    return -99999


def TFTEST(D, EPS, Reynolds = 1E6):
    print("You called the test Transmission Factor function")
    print("with arguments: D(iameter):", D, "EPS (Roughness): ", EPS, " Reynolds (number): ", '{:12.4e}'.format(Reynolds))
    return -99999
def TransmissionFactor(NAME, D, EPS = 1E-3, Reynolds = 1E6):
    """
    Usage:
    TransmissionFactor(NAME, D, EPS = 1E-3, Reynolds = 1E6)
    
    We only support the transmission factor from the Weymouth expression (for now!)
    """
    SupportedTFs = ["TFAGA","TFColebrook","TFSJain", "TFWey", "TFTEST"]
    if NAME.__name__ in SupportedTFs:
        TF = NAME(D, EPS, Reynolds = 1E6)
    else:
        print ("Unsupported Transmission Function:", NAME)
    return TF

In [2]:
D=1
print("TFWey:       ", '{:12.4e}'.format(TransmissionFactor(TFWey, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFSJain:     ", '{:12.4e}'.format(TransmissionFactor(TFSJain, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFAGA:       ", '{:12.4e}'.format(TransmissionFactor(TFAGA, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFColebrook: ", '{:12.4e}'.format(TransmissionFactor(TFColebrook, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)

D = 0.34
print("Diameter= ",D)

print("TFWey:       ", '{:12.4e}'.format(TransmissionFactor(TFWey, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFSJain:     ", '{:12.4e}'.format(TransmissionFactor(TFSJain, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFAGA:       ", '{:12.4e}'.format(TransmissionFactor(TFAGA, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)
print("TFColebrook: ", '{:12.4e}'.format(TransmissionFactor(TFColebrook, D, EPS = 1E-3, Reynolds = 1E6)), " D= ",D)

print("Validate these against the other TFs")
print("These are broadly in agreement; the AGA is high but the flow rates are similar.")
TransmissionFactor(TFTEST, 1.0, EPS = 1E-3, Reynolds = 1E6);
print("Validated for Reynolds number, now check effects of <EPS>")

TFWey:          1.0320e+01  D=  1
TFSJain:       -7.0659e+00  D=  1
TFAGA:          9.1729e+00  D=  1
TFColebrook:    7.0811e+00  D=  1
Diameter=  0.34
TFWey:          8.6214e+00  D=  0.34
TFSJain:       -6.1747e+00  D=  0.34
TFAGA:          9.1729e+00  D=  0.34
TFColebrook:    6.1826e+00  D=  0.34
Validate these against the other TFs
These are broadly in agreement; the AGA is high but the flow rates are similar.
You called the test Transmission Factor function
with arguments: D(iameter): 1.0 EPS (Roughness):  0.001  Reynolds (number):    1.0000e+06
Validated for Reynolds number, now check effects of <EPS>


#### Validation -- transport equations

In [3]:
#DEFINE Transport Equations as Qdot = f(Pin, Pout; parameters)
def WeymouthPP(Pin, Pout, D, L, Sg, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0):
    """
    WeymouthPP(Pin, Pout, D, L, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate Qdot_st from Pin, Pout, D(iameter), L(ength) etc.
    Qdot in standard cubic metres per second.
    """
    T1 = 137.2364 * (Tst/Pst)*pow(D,8/3)
    denom = L * Sg * Tavg *zavg
    T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
    Qst = T1*pow(T2,1/2)
    return Qst

def SimpTrans(Pin, Pout, D, L, Sg, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0):
    """
    SimpTrans(Pin, Pout, D, L, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Simplified  formula to generate Qdot_st from Pin, Pout, D(iameter), L(ength) etc.
    Qdot in standard cubic metres per second.
    """
    T1 = 13.283 * (Tst/Pst)*pow(D,5/2)
    denom = L * Sg * Tavg *zavg # take the transmission factor (f^{-1/2}) out
    #Does this need to be replace, maybe with Colebrook-White?
    trans = TransmissionFactor(TFWey, D)
    T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
    Qst = T1*trans*pow(T2,1/2)
    #print("ST -denom:",denom,"trans:",trans,"T1",T1,"T2",T2)
    return Qst

def AGA(Pin, Pout, D, L, Sg, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0):
    """
    AGA(Pin, Pout, D, L, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Simplified  formula to generate Qdot_st from Pin, Pout, D(iameter), L(ength) etc.
    Qdot in standard cubic metres per second.
    """
    T1 = 13.2986 * (Tst/Pst)*pow(D,5/2)
    denom = L * Sg * Tavg *zavg
    trans = TransmissionFactor(TFAGA, D)
    T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
    Qst = T1*trans*pow(T2,1/2)
    #print("AGA-denom:",denom,"trans:",trans,"T1",T1,"T2",T2)
    return Qst
def PHB(Pin, Pout, D, L, Sg, Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0):
    """
    Panhandle B(Pin, Pout, D, L, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the PHB (Coelho+Pinho Eqn 50) formula to generate Qdot_st from Pin, Pout, D(iameter), L(ength) etc.
    Qdot in standard cubic metres per second.
    """
    T1 = 135.8699 * pow(Tst/Pst,1.02)*pow(D,2.53)
    denom = L * pow(Sg,0.9608) * Tavg *zavg
    #trans = TransmissionFactor(TFAGA, D)
    T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
    Qst = T1*pow(T2,0.510)
    #print("AGA-denom:",denom,"trans:",trans,"T1",T1,"T2",T2)
    return Qst

In [4]:
ST = SimpTrans(9E6,2E6, 0.34, 1.6E5, 0.693)
print("Flow rate - ST:       ", '{:12.4e}'.format(ST),"SCM/sec", " = ",'{:12.4e}'.format((ST*24*3600)/1E6),"MSCM/day");

AGAF = AGA(9E6,2E6, 0.34, 1.6E5, 0.693)
print("Flow rate - AGA:      ", '{:12.4e}'.format(AGAF),"SCM/sec", " = ",'{:12.4e}'.format((AGAF*24*3600)/1E6),"MSCM/day");
WPP = WeymouthPP(9E6,2E6, 0.34, 1.6E5, 0.693)
print("Flow rate - Weymouth: ", '{:12.4e}'.format(WPP),"SCM/sec", " = ",'{:12.4e}'.format((WPP*24*3600)/1E6),"MSCM/day");
print(WeymouthPP.__doc__)

Flow rate - ST:          3.4745e+01 SCM/sec  =    3.0020e+00 MSCM/day
Flow rate - AGA:         3.7011e+01 SCM/sec  =    3.1978e+00 MSCM/day
Flow rate - Weymouth:    3.4786e+01 SCM/sec  =    3.0055e+00 MSCM/day

    WeymouthPP(Pin, Pout, D, L, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate Qdot_st from Pin, Pout, D(iameter), L(ength) etc.
    Qdot in standard cubic metres per second.
    


In [5]:
print("A bit of validation")
print("Weymouth")
print('{:12.3e}'.format(WeymouthPP(9E6, 2E6, 0.34, 1.65E5, 0.693)), " Base Case - Length = 165km")
print('{:12.3e}'.format(WeymouthPP(8.5E6, 2E6, 0.34, 1.65E5, 0.693))," Lower pressure drop, flow deceases.")
print('{:12.3e}'.format(WeymouthPP(8.5E6, 2.5E6, 0.34, 1.65E5, 0.693))," Lower pressure drop, flow deceases.")
print('{:12.3e}'.format(WeymouthPP(8.5E6, 2.5E6, 0.34, 1.E5, 0.693))," Shorter pipe, flow inceases.")
print("AGA")
print('{:12.3e}'.format(AGA(9E6, 2E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(AGA(8.5E6, 2E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(AGA(8.5E6, 2.5E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(AGA(8.5E6, 2.5E6, 0.34, 1.E5, 0.693)))
print("SimpTrans")
print('{:12.3e}'.format(SimpTrans(9E6, 2E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(SimpTrans(8.5E6, 2E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(SimpTrans(8.5E6, 2.5E6, 0.34, 1.65E5, 0.693)))
print('{:12.3e}'.format(SimpTrans(8.5E6, 2.5E6, 0.34, 1.E5, 0.693)))
print("Panhandle B")
print('{:12.3e}'.format(PHB(9E6, 2E6, 0.34, 1.65E5, 0.693)))

A bit of validation
Weymouth
   3.425e+01  Base Case - Length = 165km
   3.225e+01  Lower pressure drop, flow deceases.
   3.171e+01  Lower pressure drop, flow deceases.
   4.074e+01  Shorter pipe, flow inceases.
AGA
   3.645e+01
   3.431e+01
   3.374e+01
   4.334e+01
SimpTrans
   3.421e+01
   3.221e+01
   3.168e+01
   4.069e+01
Panhandle B
   4.019e+01


***
#### Utility functions

In [6]:
def InvFn(fn,Ystar, PARAMS, xmin = 0, xmax = 1E8, reltol = 1E-6, DEBUG = 0):
    """
    Given Ystar=fn(x), return Xstar s.t. f(Xstar) = Ystar
    Use linear interpolation.
    """
    # Initialise
    Xm = xmin; Xp = xmax;
    if (DEBUG == 1) : 
            print("initial: ", "Yp: ", '{:12.4e}'.format(fn(Xp, PARAMS))," Xp: ",'{:12.4e}'.format(Xp),
                  " Ym: ",'{:12.4e}'.format(fn(Xm, PARAMS))," Xm: ",'{:12.4e}'.format(Xm))
    for i in range (0,20):
        Yp = fn(Xp, PARAMS);
        Ym = abs(fn(Xm, PARAMS));
        Xstar = Xp - (Yp-Ystar)*(Xp-Xm)/(Yp-Ym)
        if not abs(Xstar) == Xstar:
            if (DEBUG == 1) : print("Warning - iter: ",i, " Xstar NOT positive: ", Xstar," replaced with 10 (to avoid zero divide)")
            Xstar = 10
        if (DEBUG == 1) : 
            print("i: ",i, "Yp: ", '{:12.4e}'.format(Yp)," Ym: ",'{:12.4e}'.format(Ym)," XStar: ",'{:12.4e}'.format(Xstar))
            print("Delta X:", '{:12.3e}'.format(abs(Xp-Xm)), " Yp: ", '{:12.2e}'.format(Yp), " Xp: ", '{:12.2e}'.format(Xp))
        Xm = Xp; Xp = Xstar; DeltaX = abs(Xp-Xm)
        if abs(DeltaX/Xp) < reltol : # this is OK if abs is very smal = 0 +complex epsilon.
            return Xstar
    print("Failure to converge!")
    print("i: ",i, "Yp: ", '{:12.4e}'.format(Yp)," Ym: ", '{:12.4e}'.format(Ym), " Xstar: ",'{:12.4e}'.format(Xstar))
    return -99999


In [7]:
# validate InvFn and WeymouthSimp1
from scipy import optimize
print("Value from WeymouthPP:  ", '{:12.3e}'.format(WeymouthPP(9.0E6,2.0E6, 0.34, 1.65E5, 0.693)));
def WeymouthSimp1(Pin, PARAMS, DEBUG = 0):
    """
    Outputs Qdot given Pin - Pout; params assumed fixed.
    """
    if DEBUG == 1 : print("WeymouthSimp1 PARAMS: ", PARAMS)
    WS1 = WeymouthPP(Pin, PARAMS[0], PARAMS[1], PARAMS[2], PARAMS[3])
    return WS1
def WeymouthSimp2(Pout, PARAMS, DEBUG = 0):
    """
    Outputs Qdot given Pout - PARAMS := [Pin,D,L,Sg]; assumed fixed.
    """
    WS2 = WeymouthPP(PARAMS[0], Pout, PARAMS[1], PARAMS[2], PARAMS[3])
    return WS2
def WeymouthSimp2p1(Pout, PARAMS):
    """
    Outputs Pout so that Qdot = Qstar by varying Pout [assume Pin is fixed] - PARAMS := [Pin,D,L,Sg,Qdot]; assumed fixed.
    """
    WS2 = WeymouthPP(PARAMS[0], Pout, PARAMS[1], PARAMS[2], PARAMS[3]) - PARAMS[4]
    return WS2
print("----")
PARAMS = [2.0E6, 0.34,1.65E5,0.693]
print("Validate WeymouthSimp1: ", '{:12.3e}'.format(WeymouthSimp1(9e6, PARAMS)))
Qdot = 34.25483928234368
print("Inlet pressure - computed via InvFn and WeymouthSimp1: ",
      '{:12.3e}'.format(InvFn(WeymouthSimp1,Qdot, PARAMS, xmin = 2e6, reltol = 1e-10, DEBUG = 0)))
IFWS = InvFn(WeymouthSimp1,Qdot, PARAMS, xmin = 1e6)
print("InvFn: ", '{:12.3e}'.format(IFWS)," should be 9.0 MPa")

print("----")
PARAMS = [9.0E6, 0.34,1.65E5,0.693]
print("Validate WeymouthSimp2: ", '{:12.3e}'.format(WeymouthSimp2(2e6, PARAMS)))
print("Inlet pressure - computed via InvFn and WeymouthSimp2: ",
      '{:12.3e}'.format(InvFn(WeymouthSimp2,34.25483928234368, PARAMS, xmax = 10e6, reltol = 1e-10, DEBUG = 0)))
print("Using fsolve")
PARAMSFS = PARAMS
PARAMSFS.append(Qdot)
FSOL = abs(optimize.fsolve(WeymouthSimp2p1, 10 , PARAMSFS, full_output=0)[0]) # we only use P^2 so -ve value is also a solution.
#Use scipy non-linear equation solver
print("FSOL - should be 2.0E6: ",'{:12.3e}'.format(FSOL))
IFWS = InvFn(WeymouthSimp2,34.25483928234368, PARAMS, xmax = 12e6)
print("InvFn: ", '{:12.3e}'.format(IFWS)," should be 2.0 MPa")

print("----")


Value from WeymouthPP:      3.425e+01
----
Validate WeymouthSimp1:     3.425e+01
Inlet pressure - computed via InvFn and WeymouthSimp1:     9.000e+06
InvFn:     9.000e+06  should be 9.0 MPa
----
Validate WeymouthSimp2:     3.425e+01
Inlet pressure - computed via InvFn and WeymouthSimp2:     2.000e+06
Using fsolve
FSOL - should be 2.0E6:     2.000e+06
InvFn:     2.000e+06  should be 2.0 MPa
----


### Networks of pipes

In [8]:
#Introduce L as an argument (not a PARAMETER) for the leaks work.
# 1) Swap the arguments 2) Validate 3) Change the parameters 4) Validate. == DONE. IS MainArgs and PARAMS used??
#DEFINE Generalised Transport Equations as RetCode = f(Qdot, Pin, Pout; parameters)
#Syntax: OUTPUT varb is initialised to -9999.
#This works because, in real life everything must be positive.
# Triple:{Qdot, Pin, Pout} legal example: {-9999, Pin>Pout, Pout>0}- find Qdot, {Qin>0,-Pin,-9999} find Pout by varying Pin
def Weymouth(Qdot, Pin, Pout, L, D, Sg, Tavg = 277.2, zavg = 1, Tst=288.15, Pst=1.01325E5, EPE=0, Const=137.2364, DEBUG = 0):
    """
    Weymouth(Qdot, Pin, Pout, L, D, Sg, Tavg = 277.2K, zavg = 1, Tst = 288.15K, Pst = 1.01325E5Pa, EPE = 0)
    - use the Weymouth formula to generate one of Qdot_st, Pin, Pout - given the other two.
    The unknown is specified by a value of -9999. 
    We use PARAMS: [D(iameter), L(ength), Sg].
    Qdot in standard cubic metres per second.
    """
    #Only one of Qdot, Pin, Pout, L should be -9999
    import math
    MainArgs = [Qdot, Pin, Pout, L]
    PARAMS = [D, Sg]
    if not (MainArgs.count(-9999) == 1): 
        print("Error - only one argument shout be -9999")
        return -9999
    T1 = 137.2364 * (Tst/Pst)*pow(D,8/3)
    denom = Sg * Tavg *zavg
    if Qdot == -9999 :
        if DEBUG == 1 : print("Weymouth to find Qdot. Pin: ", '{:12.3e}'.format(Pin), " Pout: ", '{:12.3e}'.format(Pout))
        Qst = WeymouthPP(Pin, Pout, PARAMS[0], L, PARAMS[1],
                         Tavg = 277.2, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0)
        if DEBUG == 1 : print("In Weymouth!!: ", Qst)
        return Qst
    elif Pin == -9999:
        if DEBUG == 1 : print("Weymouth to find Pin. Qdot: ", '{:12.3e}'.format(Qdot), " Pout: ", '{:12.3e}'.format(Pout))
        PinSq = L*denom*(Qdot/T1)**2 + Pout**2 + EPE
        Pin = math.sqrt(PinSq)
        if DEBUG == 1 : print("In Weymouth!!: ", Qdot,Pin)
        return Pin
    elif Pout == -9999:
        if DEBUG == 1 : print("Weymouth to find Pout. Qdot: ", '{:12.3e}'.format(Qdot), " Pin: ", '{:12.3e}'.format(Pin))
        PoutSq = Pin**2 - L*denom*(Qdot/T1)**2 + Pout**2 - EPE
        Pout = math.sqrt(PoutSq)
        return Pout
    elif L == -9999:
        T2 = (pow(Pin,2) - pow(Pout,2) - EPE)/denom
        L = (T1/Qdot)**2 *T2
        return L
    return "ERROR"


#ANSWER Weymouth(34.25483928234368 , 9E6, 2E6), L, D, Sg, zavg = 1, Tst = 288.15, Pst = 1.01325E5, EPE = 0)
print(Weymouth(-9999 , 9E6, 2E6, 1.65E5, 0.34, 0.693), " Should be: ", 34.25483928234368)#CORRECT
print(Weymouth(34.25483928234368 , 9E6, 2E6, -9999, 0.34, 0.693, DEBUG = 0), " Should be: ", 1.65E6)#CORRECT
print(Weymouth(34.25483928234368 , -9999, 2E6, 1.65E5, 0.34, 0.693, DEBUG = 0), " Should be: ", 9.0E6)#CORRECT
print(Weymouth(34.25483928234368 , 9E6, -9999, 1.65E5, 0.34, 0.693, DEBUG = 0), " Should be: ", 2.0E6)#CORRECT


34.25483928234368  Should be:  34.25483928234368
164999.99999999997  Should be:  1650000.0
9000000.0  Should be:  9000000.0
2000024.9948440643  Should be:  2000000.0


### Now for the worksheet

In [9]:
#Case 1
print(WeymouthPP(9E6, 2E6, 0.34, 1.60E5, 0.693), "SCM/sec")
print(WeymouthPP(9E6, 2E6, 0.34, 1.60E5, 0.693)*3600*24, "SCM/day")


34.78595373429907 SCM/sec
3005506.4026434394 SCM/day


In [10]:
#Case 2
def TwinPipes(TotFlow, DEBUG = 0):
    #Initialize
    Q1m = 0.5*TotFlow; Q1p = TotFlow
    #PIPE1:
    Pin1m = Weymouth(Q1m, -9999, 2E6, 1.00E5, 0.35, 0.693)
    Pin1p = Weymouth(Q1p, -9999, 2E6, 1.00E5, 0.35, 0.693)
    #PIPE2:
    Q2m = TotFlow - Q1m
    Pin2m = Weymouth(Q2m, -9999, 2E6, 1.00E5, 0.40, 0.693)
    Q2p = TotFlow - Q1p
    Pin2p = Weymouth(Q2p, -9999, 2E6, 1.00E5, 0.40, 0.693)
    if DEBUG == 1:
        print("init- "," Pin1m: ", '{:12.3e}'.format(Pin1m), " Q1m: ", '{:12.3e}'.format(Q1m))
        print("init- "," Pin2m: ", '{:12.3e}'.format(Pin2m), " Q2m: ", '{:12.3e}'.format(Q2m))
        print("init- "," Pin1p: ", '{:12.3e}'.format(Pin1p), " Q1p: ", '{:12.3e}'.format(Q1p))
        print("init- "," Pin2p: ", '{:12.3e}'.format(Pin2p), " Q2p: ", '{:12.3e}'.format(Q2p))
    
    for i in range(0,20):
        DPp = Pin1p - Pin2p; DPm = Pin1m - Pin2m;
        Q1star = Q1p - (Q1p-Q1m)*(DPp - 0)/(DPp-DPm)
        if DEBUG == 1: print("Q1p,Q1star,Q1m: ",'{:12.3e}'.format(Q1p), '{:12.3e}'.format(Q1star),'{:12.3e}'.format(Q1m),
          " Pin1p,DPp,DPm: ", '{:12.3e}'.format(Pin1p), '{:12.3e}'.format(DPp), '{:12.3e}'.format(DPm))
        Q1p = Q1m;
        Q1m = Q1star;
        #PIPE1:
        Pin1m = Weymouth(Q1m, -9999, 2E6, 1.00E5, 0.35, 0.693)
        Pin1p = Weymouth(Q1p, -9999, 2E6, 1.00E5, 0.35, 0.693)
        #PIPE2:
        Q2m = TotFlow - Q1m
        Pin2m = Weymouth(Q2m, -9999, 2E6, 1.00E5, 0.40, 0.693)
        Q2p = TotFlow - Q1p
        Pin2p = Weymouth(Q2p, -9999, 2E6, 1.00E5, 0.40, 0.693)
        if DEBUG == 1:
            print("i- ", i, " Pin1m: ", '{:12.3e}'.format(Pin1m), " Q1m: ", Q1m)
            print("i- ", i, " Pin2m: ", '{:12.3e}'.format(Pin2m), " Q2m: ", Q2m)
            print("i- ", i, " Pin1p: ", '{:12.3e}'.format(Pin1p), " Q1p: ", Q1p)
            print("i- ", i, " Pin2p: ", '{:12.3e}'.format(Pin2p), " Q2p: ", Q2p)
        if abs(DPp/Pin1p) < 10.0E-5: 
            print("TwinPipes success - Pin: ", '{:12.3e}'.format(Pin1p))
            return Pin1p

TwP = TwinPipes(6.0E6/(3600*24))
print("Output from TwinPipes - inlet pressure: ", '{:12.3e}'.format(TwP))
print("Weymouth Pipe 1 - Flow rate: ", '{:12.3e}'.format(24*3600*WeymouthPP(TwP, 2E6, 0.35, 1.00E5, 0.693)))
print("Weymouth Pipe 2 - Flow rate: ", '{:12.3e}'.format(24*3600*WeymouthPP(TwP, 2E6, 0.40, 1.00E5, 0.693)))
print("Total DAILY flow rate: ", 
      '{:12.3e}'.format(24*3600*(WeymouthPP(TwP, 2E6, 0.35, 1.00E5, 0.693) + WeymouthPP(TwP, 2E6, 0.40, 1.00E5, 0.693))))


TwinPipes success - Pin:     5.646e+06
Output from TwinPipes - inlet pressure:     5.646e+06
Weymouth Pipe 1 - Flow rate:     2.471e+06
Weymouth Pipe 2 - Flow rate:     3.529e+06
Total DAILY flow rate:     6.000e+06


In [11]:
#Case 3 :)
#Case 3.1 - Assume everything is at the same temperature and height.

def ThreePipes(TotFlow, DEBUG = 0):
    #Initialize
    Pmidp = 2*8.5E6/3+2.0E6/3
    Pmidm = 8.5E6/3 + 2*2.0E6/3
    #PIPE1:
    Q1m = Weymouth(-9999, 8.5E6, Pmidm, 1.00E5, 0.35, 0.693)
    Q1p = Weymouth(-9999, 8.5E6, Pmidp, 1.00E5, 0.35, 0.693)
    #PIPE2:
    Q2m = Weymouth( -9999, 8.5E6, Pmidm, 1.00E5, 0.40, 0.693)
    Q2p = Weymouth( -9999, 8.5E6, Pmidp, 1.00E5, 0.40, 0.693)
    #PIPE3:
    Q3m = Weymouth( -9999, Pmidm, 2E6, 1.50E5, 0.40, 0.693)
    Q3p = Weymouth( -9999, Pmidp, 2E6, 1.50E5, 0.40, 0.693)

    for i in range(1,20):
        if DEBUG == 1:
            print("i- ",i," Pmid1m: ", '{:12.3e}'.format(Pmidm), " Q1m: ", '{:12.3e}'.format(Q1m))
            print("i- ",i," Pmid2m: ", '{:12.3e}'.format(Pmidm), " Q2m: ", '{:12.3e}'.format(Q2m))
            print("i- ",i," Pmid1p: ", '{:12.3e}'.format(Pmidp), " Q1p: ", '{:12.3e}'.format(Q1p))
            print("i- ",i," Pmid2p: ", '{:12.3e}'.format(Pmidp), " Q2p: ", '{:12.3e}'.format(Q2p))
            print("i- ",i," Pmid3m: ", '{:12.3e}'.format(Pmidm), " Q3p: ", '{:12.3e}'.format(Q3m))
            print("i- ",i," Pmid3p: ", '{:12.3e}'.format(Pmidp), " Q3p: ", '{:12.3e}'.format(Q3p))
        #First calculate the flow rate through the twin pipes and the outlet pipe (3):
        Q1m = Weymouth(-9999, 8.5E6, Pmidm, 1.00E5, 0.35, 0.693)
        Q2m = Weymouth(-9999, 8.5E6, Pmidm, 1.00E5, 0.40, 0.693)
        Q3m = Weymouth(-9999, Pmidm, 2.0E6, 1.50E5, 0.50, 0.693)
        ErrQm = Q3m - ( Q1m + Q2m)
        Q1p = Weymouth(-9999, 8.5E6, Pmidp, 1.00E5, 0.35, 0.693)
        Q2p = Weymouth(-9999, 8.5E6, Pmidp, 1.00E5, 0.40, 0.693)
        Q3p = Weymouth(-9999, Pmidp, 2.0E6, 1.50E5, 0.50, 0.693)
        ErrQp = Q3p - ( Q1p + Q2p)
        PmidStar = Pmidm - (Pmidm - Pmidp)*(ErrQm - 0)/(ErrQm - ErrQp)
        Pmidm = Pmidp
        Pmidp = PmidStar
        if abs(ErrQp/Q3p) < 10.0E-5: 
            print("ThreePipes success - Pmid: ", '{:12.3e}'.format(Pmidp))
            return Pmidp
    return "ERROR"



T3P = ThreePipes(6.0E6/(3600*24), DEBUG = 0)
print("Output from ThreePipes - mid-manifold pressure: ", '{:12.3e}'.format(T3P))
print("Weymouth Pipe 1 - Flow rate: ", 24*3600*WeymouthPP(8.5E6, T3P, 0.35, 1.00E5, 0.693))
print("Weymouth Pipe 2 - Flow rate: ", 24*3600*WeymouthPP(8.5E6, T3P, 0.40, 1.00E5, 0.693))
print("Weymouth Pipe 3 - Flow rate: ", 24*3600*WeymouthPP(T3P, 2.0E6, 0.50, 1.50E5, 0.693))

print("Total INPUT to mid-manifold flow rate: ", 
      24*3600*((WeymouthPP(8.5E6, T3P, 0.35, 1.00E5, 0.693) + WeymouthPP(8.5E6, T3P, 0.40, 1.00E5, 0.693))))


ThreePipes success - Pmid:     6.544e+06
Output from ThreePipes - mid-manifold pressure:     6.544e+06
Weymouth Pipe 1 - Flow rate:  2539072.6294731856
Weymouth Pipe 2 - Flow rate:  3625102.709359314
Weymouth Pipe 3 - Flow rate:  6164175.339554842
Total INPUT to mid-manifold flow rate:  6164175.338832499


In [19]:
#A temp cell to develop stuf fin
